# Kaggle dataset preprocessing

#### Imports

In [1]:
#pip install opencv-python

In [2]:
import cv2
import os
import numpy as np


In [23]:
from tensorflow.keras.utils import image_dataset_from_directory

import warnings
warnings.filterwarnings("ignore")

2024-03-10 04:55:54.495789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading dataset

🚨 This will only be needed for the training because for the predicting the image will be loaded/stored differently

❓ @Susi : As the Kaggle dataset images are classified in folders named by its class, maybe is more convinient to load them as Tensorflow Datasets as we saw in the CNN challenges so that it is easier to handle them when training (making batches, etc)

### Ainhoa's code

In [15]:
# current directory from where the notebook is run
!pwd

/Users/Ainhoa/code/kirianpg/waste_sorter_smart_bin/notebooks


In [21]:
# data directory where the Garbage classification images folder is stored
data_dir = '../raw_data/Garbage classification'

# check by listing all the elements in the directory
os.listdir(data_dir)

['paper', '.DS_Store', 'metal', 'cardboard', 'trash', 'glass', 'plastic']

In [24]:
# Load data as Tensorflow Dataset
batch_size = None

data = image_dataset_from_directory(
  data_dir, # Folder directory
  labels = "inferred", # inferred from sub folder name
  label_mode = "categorical",
  seed=None,
  image_size=(224, 224), # we can resize directly the images
  batch_size=batch_size)

# With Tensorflow Dataset object we can also define directly the training/validating

Found 2527 files belonging to 6 classes.


2024-03-10 04:56:31.286200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# check class names
data.class_names

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [26]:
data.element_spec

(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(6,), dtype=tf.float32, name=None))

In [20]:
#data.file_paths

In [37]:
type(data)

tensorflow.python.data.ops.dataset_ops.ShuffleDataset

In [38]:
for X,y in data:
    print(X.shape)

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [39]:
X.shape

TensorShape([224, 224, 3])

### Susi's code

❓ @Susi :
- I think we need to "generalized" the whole data path => maybe put it as a env variable?
- I think that the CNN input are tensors... if we are using this kind of model, we should built a tensor (number of images, height, width, channels)
- Maybe instead of using cv2 library (which needs to install) skimage

In [3]:
def load_images_with_labels(root_folder):
    images_with_labels = []
    for object_folder in os.listdir(root_folder):
        object_path = os.path.join(root_folder, object_folder)
        if os.path.isdir(object_path):
            for filename in os.listdir(object_path):
                img_path = os.path.join(object_path, filename)
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        label = object_folder
                        images_with_labels.append((img, label))
    return images_with_labels

# Define root folder containing subfolders for each object category
current_directory = os.getcwd()
root_folder = os.path.dirname(current_directory) + '/raw_data/Garbage classification'

# Load images with labels
images_with_labels = load_images_with_labels(root_folder)


In [12]:
current_directory

'/Users/Ainhoa/code/kirianpg/waste_sorter_smart_bin/notebooks'

In [13]:
root_folder

'/Users/Ainhoa/code/kirianpg/waste_sorter_smart_bin/raw_data/Garbage classification'

In [19]:
images_with_labels[0][1]

'paper'

## Preprocessing

🚨 The preprocessing is going to depend a bit in the architecture of the model (some include already the preprocessing) and, if we use transfer learning, we might have to use the specific preprocessing used in the pretrained model

In [10]:
def preprocess_image(img, target_size):
    # Resize image
    img_resized = cv2.resize(img, target_size)

    # Normalize pixel values
    img_normalized = img_resized / 255.0

    return img_normalized

# Define target size for resizing
target_size = (224, 224)

# Preprocess each image in the dataset
preprocessed_data = []
for img, label in images_with_labels:
    preprocessed_img = preprocess_image(img, target_size)
    preprocessed_data.append((preprocessed_img, label))

# Convert preprocessed_data to NumPy arrays
X = np.array([data[0] for data in preprocessed_data])
y = np.array([data[1] for data in preprocessed_data])
X


array([[[[0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         ...,
         [0.74117647, 0.8       , 0.83137255],
         [0.7372549 , 0.79607843, 0.83137255],
         [0.73333333, 0.79215686, 0.82745098]],

        [[0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         ...,
         [0.7372549 , 0.79607843, 0.83137255],
         [0.73333333, 0.79215686, 0.82745098],
         [0.73333333, 0.79215686, 0.82745098]],

        [[0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         ...,
         [0.73333333, 0.79215686, 0.82745098],
         [0.73333333, 0.79215686, 0.82745098],
         [0.72941176, 0.78823529, 0.82352941]],

        ...,

        [[0.56470588, 0.42745098, 0.44313725],
         [0.63137255, 0.49411765, 0.50980392]

## Code for the .py files in "logic"

🚨 Here we could just sum up the pieces of code that will we used in the logil of the app if the notebook gets messy

## CNN model

🚨 I think we should either decide in a first base model and give it to the guys that will do the modeling part, either talk with the team and decide all together which kind of model we will use (at least, the first layer type, so that we can adapt the preprocessing to it)

In [ ]:
# Define the CNN model
def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (224, 224, 3)
num_classes = len(set(y))

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f"Test Accuracy: {accuracy}")
